In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.dm import DFTemporalDataModule

trans = {
	'HorizontalFlip': {'p': 0.5},
	'VerticalFlip': {'p': 0.5},
	'RandomRotate90': {'p': 0.5},
	'Transpose': {'p': 0.5},
	# 'RandomResizedCrop': {'height': 256, 'width': 256, 'scale': (0.8, 1.0), 'ratio': (0.75, 1.3333333333333333), 'p': 0.5},
	# 'ShiftScaleRotate': {'shift_limit': 0.2, 'scale_limit': 0.2, 'rotate_limit': 30, 'p': 0.2},
}

dm = DFTemporalDataModule(batch_size=1, use_clouds=True, s2_bands=(2,1,0), train_trans=trans)
# dm = DFTemporalDataModule(batch_size=1, train_trans=trans)

dm.setup()

In [7]:
import pytorch_lightning as pl
from src.module2 import Transformer
import torch

dm = DFTemporalDataModule(batch_size=1)

hparams = {
	'encoder': 'resnet18',
	'pretrained': 'imagenet',
	'seq_len': 12,
	'in_channels_s1': 2,
	'in_channels_s2': 3,
    'embed_dim': 256,
    'n_heads': 4,
    'num_blocks': 2,
	'optimizer': 'Adam',
	'optimizer_params': {
		'lr': 1e-3
	},
	# 'scheduler': {
	# 	'OneCycleLR': {
	# 		'max_lr': 1e-2,
	# 		'pct_start': 0.1,
	# 		'total_steps': 1000,
	# 		# 'verbose': True
	# 	}
	# }
}

module = Transformer(hparams)

trainer = pl.Trainer(
	gpus=1,
	precision=16,
	overfit_batches=1,
	max_epochs=300,
	logger=None,
	enable_checkpointing=False,
)

trainer.fit(module, dm)

: 

: 

In [2]:
# initialize a raster of the size of the output space
# tile with positional encoding (sin, cos) 
# encode with MLP in a set of query vectors

In [9]:
import torch 

# esto es lo que quiero a la salida (será parámtero y tendrá positional encoding)

a = torch.randn(16, 1, 256, 256)
a.shape

torch.Size([16, 1, 256, 256])

In [10]:
import torch.nn as nn

# https://github.com/jankrepl/mildlyoverfitted/blob/master/github_adventures/vision_transformer/custom.py

class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_chans, embed_dim):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (B, E, P, P)
        x = x.flatten(2)  # (B, E, N)
        x = x.transpose(1, 2)  # (B, N, E)
        return x

In [16]:
pe = PatchEmbedding(256, 8, 1, 256)

b = pe(a) # estas son mis queries
b.shape

torch.Size([16, 1024, 256])

In [17]:
# cada imagen y multi-scale feature emite su key-value

In [18]:
# con unet

mf = [
    torch.randn(16, 64, 128, 128),
    torch.randn(16, 64, 64, 64),
    torch.randn(16, 128, 32, 32),
    torch.randn(16, 256, 16, 16),
    torch.randn(16, 512, 8, 8)
]

In [19]:
mes = [
    PatchEmbedding(128, 8, 64, 256),
    PatchEmbedding(64, 8, 64, 256),
    PatchEmbedding(32, 8, 128, 256),
    PatchEmbedding(16, 8, 256, 256),
    PatchEmbedding(8, 8, 512, 256)
]

cs = [me(m) for me, m in zip(mes, mf)]
for c in cs:
    print(c.shape)

c = torch.cat(cs, dim=1)
c.shape

torch.Size([16, 256, 256])
torch.Size([16, 64, 256])
torch.Size([16, 16, 256])
torch.Size([16, 4, 256])
torch.Size([16, 1, 256])


torch.Size([16, 341, 256])

In [20]:
# repetir para serie temporal 
# añadir los correspondientes positional, sensor, etc encodings

c2 = torch.cat([c]*12*2, dim=1)
c2.shape

torch.Size([16, 8184, 256])

In [27]:
from src.attention import Block

block = Block(256, 256)
d = block(c, b)
d.shape

torch.Size([16, 1024, 256])

In [29]:
# retrieve image

from einops import rearrange

e = rearrange(d, 'b n (h w) -> b n h w', h=16, w=16)
e.shape

torch.Size([16, 1024, 16, 16])

In [31]:
unproj = nn.Sequential(
    nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
    nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
    nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
    nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
    # nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
    nn.Conv2d(64, 1, kernel_size=1, stride=1)
)

f = unproj(e)
f.shape

torch.Size([16, 1, 256, 256])

In [35]:
# con bifpn

mf = [
    torch.randn(16, 64, 128, 128),
    torch.randn(16, 64, 64, 64),
    torch.randn(16, 64, 32, 32),
    torch.randn(16, 64, 16, 16),
    torch.randn(16, 64, 8, 8)
]

In [36]:
mes = [
    PatchEmbedding(128, 8, 64, 64),
    PatchEmbedding(64, 8, 64, 64),
    PatchEmbedding(32, 8, 64, 64),
    PatchEmbedding(16, 8, 64, 64),
    PatchEmbedding(8, 8, 64, 64)
]

cs = [me(m) for me, m in zip(mes, mf)]
for c in cs:
    print(c.shape)

c = torch.cat(cs, dim=1)
c.shape

torch.Size([16, 256, 64])
torch.Size([16, 64, 64])
torch.Size([16, 16, 64])
torch.Size([16, 4, 64])
torch.Size([16, 1, 64])


torch.Size([16, 341, 64])

In [37]:
# con bifpn, si tengo dos sensores, cada sensor tiene 12 imágenes, ...

mf = [
    torch.randn(16, 2, 12, 64, 128, 128),
    torch.randn(16, 2, 12, 64, 64, 64),
    torch.randn(16, 2, 12, 64, 32, 32),
    torch.randn(16, 2, 12, 64, 16, 16),
    torch.randn(16, 2, 12, 64, 8, 8)
]

In [38]:


class MultiscaleFeatureEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_chans, embed_dim):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = rearrange(x, 'b s l c h w -> b (s l c) h w')
        x = self.proj(x)  # (B, E, P, P)
        x = x.flatten(2)  # (B, E, N)
        x = x.transpose(1, 2)  # (B, N, E)
        return x

In [39]:
mes = [
    MultiscaleFeatureEmbedding(128, 8, 64*2*12, 64),
    MultiscaleFeatureEmbedding(64, 8, 64*2*12, 64),
    MultiscaleFeatureEmbedding(32, 8, 64*2*12, 64),
    MultiscaleFeatureEmbedding(16, 8, 64*2*12, 64),
    MultiscaleFeatureEmbedding(8, 8, 64*2*12, 64)
]

cs = [me(m) for me, m in zip(mes, mf)]
for c in cs:
    print(c.shape)

torch.Size([16, 256, 64])
torch.Size([16, 64, 64])
torch.Size([16, 16, 64])
torch.Size([16, 4, 64])
torch.Size([16, 1, 64])
